In [50]:
import numpy as np
import torch

In [51]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device('cpu')

print(device)

In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [52]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [53]:
from torch.utils.data import DataLoader


tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)

eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [58]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [59]:
from torch.optim import AdamW
from transformers import get_scheduler


optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [60]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

In [57]:
from tqdm.auto import tqdm
from datasets import load_metric

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        
    model.eval()
    acc_metric = load_metric("accuracy")
    f1_metric = load_metric("f1")
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            acc_metric.add_batch(predictions=predictions, references=batch["labels"])
            f1_metric.add_batch(predictions=predictions, references=batch["labels"])

    print(f1_metric.compute(average="macro"), acc_metric.compute())

In [62]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support


def train(model, optimizer, lr_scheduler, train_loader, eval_loader, num_epochs):
    progress_bar = tqdm(range(num_training_steps))
    
    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            progress_bar.update(1)
            
        model.eval()
        labels_list = []
        preds_list = []
        with torch.no_grad():
            for batch in eval_loader:
                batch = {k: v.to(device) for k, v in batch.items()}

                outputs = model(**batch)
                _, preds = torch.max(outputs.logits, dim=1, keepdim=False)
                labels_list.extend(batch["labels"].cpu().numpy().tolist())
                preds_list.extend(preds.cpu().numpy().tolist())
                
        metrics = compute_metrics(labels_list, preds_list)
        print(f"valid accuracy: {metrics['accuracy']}\n"
              f"valid f1: {metrics['f1']}\n")


def compute_metrics(y_true, y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro")
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": confusion_matrix(y_true, y_pred)
    }


train(model, optimizer, lr_scheduler, train_dataloader, eval_dataloader, num_epochs)